In [1]:
import pandas as pd

In [3]:
import json
import os

file_path = "D:\\AI\\GenAI\\my_projects\\RAG-based-AI-assistant-for-Central-Bank-of-Sri-Lanka\\data\\processed\\AR_2020_presentation_e.json"

with open(file_path, "r", encoding="utf-8") as f:
    pages = json.load(f)

print(len(pages))
print(pages[0].keys())

62
dict_keys(['page_number', 'text'])


In [4]:
full_text = ""

for page in pages:
    full_text += page["text"] + "\n"

print(full_text[:1500])

The Statutory Requirement
Section 35 of the Monetary Law Act:
…Within four months after the end of each financial 
year, the Monetary Board shall submit to the 
Minister in charge of the subject of Finance and shall 
publish an annual report on the condition of the 
Central Bank and a review of the policies and 
measures adopted by the Monetary Board during 
the financial year and an analysis of the economic 
and financial circumstances which prompted those 
policies and measures…
This is the 71st Annual Report of the 
Monetary Board of the Central bank for 
the Year 2020
2
Content of the Annual Report
Volume I Part I - State of the economy, its performance, policies and issues
Statistical Appendix
Special Statistical Appendix
Volume II Part II - Accounts and Operations of the Central Bank 
Part III - Administrative measures adopted by the Government and the 
Monetary Board during the year relating to the functions and 
operations of the Central Bank and banking institutions in Sri 
La

In [5]:
import re

def split_by_sections(text):
    pattern = r"(Section\s+\d+.*?|Regulation\s+\d+.*?|Article\s+\w+.*?)(?=Section\s+\d+|Regulation\s+\d+|Article\s+\w+|$)"
    
    matches = re.findall(pattern, text, flags=re.DOTALL)
    
    return matches

sections = split_by_sections(full_text)

print("Total sections found:", len(sections))
print(sections[0][:1000])

Total sections found: 1
Section 35 of the Monetary Law Act:
…Within four months after the end of each financial 
year, the Monetary Board shall submit to the 
Minister in charge of the subject of Finance and shall 
publish an annual report on the condition of the 
Central Bank and a review of the policies and 
measures adopted by the Monetary Board during 
the financial year and an analysis of the economic 
and financial circumstances which prompted those 
policies and measures…
This is the 71st Annual Report of the 
Monetary Board of the Central bank for 
the Year 2020
2
Content of the Annual Report
Volume I Part I - State of the economy, its performance, policies and issues
Statistical Appendix
Special Statistical Appendix
Volume II Part II - Accounts and Operations of the Central Bank 
Part III - Administrative measures adopted by the Government and the 
Monetary Board during the year relating to the functions and 
operations of the Central Bank and banking institutions in Sri 
Lank

In [6]:
def extract_section_number(section_text):
    match = re.search(r"(Section\s+\d+(\.\d+)*)", section_text)
    if match:
        return match.group(1)
    return "Unknown"

smart_chunks = []

for i, section in enumerate(sections):
    smart_chunks.append({
        "text": section.strip(),
        "section_number": extract_section_number(section),
        "chunk_id": i
    })

smart_chunks[0]

{'text': "Section 35 of the Monetary Law Act:\n…Within four months after the end of each financial \nyear, the Monetary Board shall submit to the \nMinister in charge of the subject of Finance and shall \npublish an annual report on the condition of the \nCentral Bank and a review of the policies and \nmeasures adopted by the Monetary Board during \nthe financial year and an analysis of the economic \nand financial circumstances which prompted those \npolicies and measures…\nThis is the 71st Annual Report of the \nMonetary Board of the Central bank for \nthe Year 2020\n2\nContent of the Annual Report\nVolume I Part I - State of the economy, its performance, policies and issues\nStatistical Appendix\nSpecial Statistical Appendix\nVolume II Part II - Accounts and Operations of the Central Bank \nPart III - Administrative measures adopted by the Government and the \nMonetary Board during the year relating to the functions and \noperations of the Central Bank and banking institutions in Sr

In [7]:
from datetime import datetime

document_name = "monetary_policy_2024"
regulation_type = "Monetary_Policy"
document_date = "2024"

final_chunks = []

for chunk in smart_chunks:
    metadata = {
        "document_name": document_name,
        "regulation_type": regulation_type,
        "date": document_date,
        "section_number": chunk["section_number"],
        "chunk_id": chunk["chunk_id"]
    }

    final_chunks.append({
        "text": chunk["text"],
        "metadata": metadata
    })

final_chunks[0]

{'text': "Section 35 of the Monetary Law Act:\n…Within four months after the end of each financial \nyear, the Monetary Board shall submit to the \nMinister in charge of the subject of Finance and shall \npublish an annual report on the condition of the \nCentral Bank and a review of the policies and \nmeasures adopted by the Monetary Board during \nthe financial year and an analysis of the economic \nand financial circumstances which prompted those \npolicies and measures…\nThis is the 71st Annual Report of the \nMonetary Board of the Central bank for \nthe Year 2020\n2\nContent of the Annual Report\nVolume I Part I - State of the economy, its performance, policies and issues\nStatistical Appendix\nSpecial Statistical Appendix\nVolume II Part II - Accounts and Operations of the Central Bank \nPart III - Administrative measures adopted by the Government and the \nMonetary Board during the year relating to the functions and \noperations of the Central Bank and banking institutions in Sr

In [8]:
from datetime import datetime

document_name = "monetary_policy_2024"
regulation_type = "Monetary_Policy"
document_date = "2024"

final_chunks = []

for chunk in smart_chunks:
    metadata = {
        "document_name": document_name,
        "regulation_type": regulation_type,
        "date": document_date,
        "section_number": chunk["section_number"],
        "chunk_id": chunk["chunk_id"]
    }

    final_chunks.append({
        "text": chunk["text"],
        "metadata": metadata
    })

final_chunks[0]

{'text': "Section 35 of the Monetary Law Act:\n…Within four months after the end of each financial \nyear, the Monetary Board shall submit to the \nMinister in charge of the subject of Finance and shall \npublish an annual report on the condition of the \nCentral Bank and a review of the policies and \nmeasures adopted by the Monetary Board during \nthe financial year and an analysis of the economic \nand financial circumstances which prompted those \npolicies and measures…\nThis is the 71st Annual Report of the \nMonetary Board of the Central bank for \nthe Year 2020\n2\nContent of the Annual Report\nVolume I Part I - State of the economy, its performance, policies and issues\nStatistical Appendix\nSpecial Statistical Appendix\nVolume II Part II - Accounts and Operations of the Central Bank \nPart III - Administrative measures adopted by the Government and the \nMonetary Board during the year relating to the functions and \noperations of the Central Bank and banking institutions in Sr

In [10]:
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200
)

final_documents = []

for chunk in final_chunks:
    smaller_chunks = splitter.split_text(chunk["text"])
    
    for i, small_chunk in enumerate(smaller_chunks):
        metadata = chunk["metadata"].copy()
        metadata["sub_chunk_id"] = i
        
        final_documents.append({
            "text": small_chunk,
            "metadata": metadata
        })

print("Final total chunks:", len(final_documents))

d:\AI\GenAI\my_projects\RAG-based-AI-assistant-for-Central-Bank-of-Sri-Lanka\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Final total chunks: 42


In [11]:
final_documents

[{'text': 'Section 35 of the Monetary Law Act:\n…Within four months after the end of each financial \nyear, the Monetary Board shall submit to the \nMinister in charge of the subject of Finance and shall \npublish an annual report on the condition of the \nCentral Bank and a review of the policies and \nmeasures adopted by the Monetary Board during \nthe financial year and an analysis of the economic \nand financial circumstances which prompted those \npolicies and measures…\nThis is the 71st Annual Report of the \nMonetary Board of the Central bank for \nthe Year 2020\n2\nContent of the Annual Report\nVolume I Part I - State of the economy, its performance, policies and issues\nStatistical Appendix\nSpecial Statistical Appendix\nVolume II Part II - Accounts and Operations of the Central Bank \nPart III - Administrative measures adopted by the Government and the \nMonetary Board during the year relating to the functions and \noperations of the Central Bank and banking institutions in S